# 3. Construcción de Agentes con LangChain

## Objetivos de Aprendizaje
- Comprender las ventajas de usar un framework como LangChain para el desarrollo de agentes.
- Aprender a definir herramientas (Tools) de forma sencilla con el decorador `@tool`.
- Crear un agente utilizando el constructor `create_openai_tools_agent`.
- Ejecutar el agente y gestionar la interacción mediante el `AgentExecutor`.

## ¿Por qué usar LangChain?

En los notebooks anteriores, construimos agentes desde cero. Primero, parseando texto, y luego, usando `function calling` nativo. Aunque el `function calling` es una gran mejora, todavía teníamos que:

1.  **Gestionar el historial de mensajes (`messages`) manualmente**: Añadir cada respuesta del usuario, del asistente y de la herramienta a la lista.
2.  **Orquestar el flujo de llamadas**: Escribir la lógica `if/else` para decidir si llamar a una herramienta, ejecutarla y volver a llamar al modelo.
3.  **Formatear las herramientas**: Escribir el JSON Schema para cada herramienta, lo cual es propenso a errores.

**LangChain** es un framework que abstrae toda esta complejidad. Actúa como una capa intermedia que simplifica enormemente la creación de aplicaciones basadas en LLMs, incluyendo los agentes. 

**Ventajas clave:**
- **Componentes Modulares**: Ofrece piezas reutilizables (LLMs, Prompts, Herramientas, etc.) que se pueden ensamblar fácilmente.
- **Agentes Listos para Usar**: Proporciona abstracciones de alto nivel como el `AgentExecutor` que manejan el ciclo de razonamiento (ReAct) por nosotros.
- **Integraciones**: Se conecta con cientos de fuentes de datos, APIs y otros servicios de forma nativa.

### 1. Instalación y Configuración

In [ ]:
!pip install langchain langchain-openai openai wikipedia -q

In [1]:
import os
import wikipedia
from langchain_openai import ChatOpenAI

# Configurar el idioma de Wikipedia
wikipedia.set_lang('es')

# --- Configuración del LLM con LangChain ---
# LangChain actúa como un "envoltorio" (wrapper) sobre el cliente de OpenAI
try:
    llm = ChatOpenAI(
        model="gpt-4o",
        openai_api_base=os.environ.get("GITHUB_BASE_URL"),
        openai_api_key=os.environ.get("GITHUB_TOKEN"),
        temperature=0
    )
    print("✅ LLM de LangChain configurado.")
except Exception as e:
    print(f"❌ Error configurando el LLM: {e}")
    llm = None

✅ LLM de LangChain configurado.


### 2. Definición de Herramientas con LangChain

LangChain simplifica enormemente la creación de herramientas. En lugar de escribir un JSON Schema manual, simplemente usamos el decorador `@tool` sobre una función de Python. LangChain se encarga de inferir el esquema a partir de la firma de la función y su docstring.

El docstring es **muy importante**, ya que se usa como la descripción que el LLM ve para decidir si usar la herramienta o no.

In [2]:
from langchain.agents import tool

@tool
def get_wikipedia_summary(query: str) -> str:
    """Busca en Wikipedia un tema y devuelve un resumen de 2 frases. Es ideal para obtener información sobre personas, lugares o conceptos históricos y científicos."""
    try:
        summary = wikipedia.summary(query, sentences=2)
        return summary
    except wikipedia.exceptions.PageError:
        return f"No se encontró ninguna página para '{query}'."
    except wikipedia.exceptions.DisambiguationError as e:
        return f"La búsqueda para '{query}' es ambigua. Opciones: {e.options[:3]}"

tools = [get_wikipedia_summary]

print("✅ Herramientas de LangChain definidas.")

✅ Herramientas de LangChain definidas.


### 3. Creación del Agente

Para crear el agente, necesitamos dos cosas:

1.  **Un Prompt**: Una plantilla que le dice al agente cómo razonar y cómo usar las herramientas. LangChain ya tiene prompts pre-construidos y optimizados para esto. Usaremos `hub.pull("hwchase17/openai-functions-agent")` para obtener una plantilla probada.
2.  **El Agente en sí**: Usamos la función `create_openai_tools_agent`, que une el LLM, las herramientas y el prompt.

El resultado es un `Runnable` de LangChain, que es el agente listo para ser ejecutado.

In [3]:
from langchain import hub
from langchain.agents import create_openai_tools_agent

# Descargar un prompt pre-diseñado y optimizado para agentes con function calling
prompt = hub.pull("hwchase17/openai-tools-agent")

# Unir el LLM, las herramientas y el prompt para crear el agente
agent = create_openai_tools_agent(llm, tools, prompt)

print("✅ Agente de LangChain creado.")

✅ Agente de LangChain creado.


### 4. Ejecución del Agente con `AgentExecutor`

El `agent` que creamos en el paso anterior es solo el "cerebro". Sabe cómo razonar, pero no puede ejecutar el ciclo de `Pensamiento -> Acción -> Observación` por sí mismo.

Para eso, usamos el **`AgentExecutor`**. Esta clase toma el agente y las herramientas, y se encarga de toda la orquestación:

- Llama al agente con la entrada del usuario.
- Si el agente decide usar una herramienta, el `AgentExecutor` la ejecuta.
- Pasa el resultado de la herramienta de vuelta al agente.
- Repite el proceso hasta que el agente da una respuesta final.
- Gestiona el historial de la conversación (`chat_history`).

In [4]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("✅ AgentExecutor listo para funcionar.")

✅ AgentExecutor listo para funcionar.


### 5. Invocando al Agente

Ahora, simplemente llamamos al método `invoke` del `agent_executor` con la pregunta. El parámetro `chat_history` es opcional pero útil para conversaciones de seguimiento.

In [5]:
query = "¿Quién fue Marie Curie y cuáles fueron sus logros más importantes?"

response = agent_executor.invoke({
    "input": query,
    "chat_history": []
})

print(f"🏁 Respuesta Final del Agente: {response}")



> Entering new AgentExecutor chain...

Invoking: `get_wikipedia_summary` with `{'query': 'Marie Curie'}`


Maria Salomea Skłodowska-Curie,[A]​[B]​ más conocida como Marie Curie[C]​[B]​ o Madame Curie (Varsovia, 7 de noviembre de 1867-Passy, 4 de julio de 1934), fue una física y química de origen polaco. Pionera en el campo de la radiactividad, es la primera y única persona en recibir dos premios Nobel en distintas especialidades científicas: Física y Química.[D]​ También fue la primera mujer en ocupar el puesto de profesora en la Universidad de París y la primera en recibir sepultura con honores en el Panteón de París por méritos propios en 1995.[E]​
Nació en Varsovia, en lo que entonces era el Zarato de Polonia (territorio administrado por el Imperio ruso).Marie Curie fue una física y química polaca, pionera en el campo de la radiactividad, y la primera persona en recibir dos premios Nobel en distintas disciplinas científicas: Física y Química. Entre sus logros más importantes desta

## Conclusiones

Como hemos visto, LangChain reduce drásticamente la cantidad de código repetitivo y la complejidad de construir un agente. Nos hemos podido centrar en:

1.  **Definir la lógica de la herramienta**: La función `get_wikipedia_summary`.
2.  **Ensamblar los componentes**: Unir el LLM, las herramientas y un prompt usando las abstracciones de LangChain.

El `AgentExecutor` se encargó de todo el ciclo de ejecución, el manejo de estado y la orquestación, que antes teníamos que programar manualmente.

En el siguiente notebook, exploraremos **CrewAI**, otro framework de alto nivel que se especializa en la creación de equipos de agentes que colaboran para resolver tareas aún más complejas.